In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
Home = os.getcwd()
DataFolder = os.path.join(Home,'Data')
df = pd.read_csv(os.path.join(DataFolder,'MyDataSet.csv'))
df_cols = list(df.columns)
df = df[df.filter(regex='^(?!Unnamed)').columns]

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from niapy.algorithms import basic

In [4]:
def Define_Model():
    model = RandomForestClassifier()
    param_grid = {}
    return model, param_grid

In [5]:
df_cols = list(df.columns)
y = df[df_cols[-1]]
X = df[df_cols[:-1]]
start_val = 0
model, param_grid = Define_Model()

In [ ]:
def SKLEARN_CUCKOOSEARCH(Kmodel, Kparam_grid):
    lst = []
    for Kpopulation_size in range(10,101,10): #10
        for Kpa in np.arange(0.01,1.01,.01): #100
            
            Algo = basic.cs.CuckooSearch()
            Algo.set_parameters(
                population_size = Kpopulation_size,
                pa = Kpa
            )
            NIS = NatureInspiredSearchCV(
                estimator = Kmodel,
                param_grid = Kparam_grid,
                algorithm=Algo
            )
            lst.append(NIS)
    return lst

In [6]:
def Save_Often(CS_Dict,Name):
    ResultFolder = os.path.join(Home, 'Result')
    Name = os.path.join(ResultFolder, Name)
    with open(Name,'w') as kanaishk:
        json.dump(CS_Dict,kanaishk)

In [ ]:
NIS_lst = SKLEARN_CUCKOOSEARCH(model, param_grid)
print(len(NIS_lst))
Best_Score = 0
Best_Para = None 
CS_Dict = dict()

In [ ]:
for i in range(len(NIS_lst)):
    if(i%100==0 and i != start_val):
        Name = f'Kanaishk Cuckoo Search Dict-{i/100}.json'
        Save_Often(CS_Dict,Name)
        CS_Dict = dict()
    Curr_Candidate = NIS_lst[i]
    Curr_Candidate.fit(X,y)
    Curr_Para = NIS_lst[i].algorithm.get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    CS_Dict[str(Curr_Para)] = Curr_Score
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para

In [ ]:
#Incase some of the model do not work
'''CS_Dict = dict()
for i in range(800,900):
    Curr_Candidate = NIS_lst[i]
    Curr_Candidate.fit(X,y)
    Curr_Para = NIS_lst[i].algorithm.get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    CS_Dict[str(Curr_Para)] = Curr_Score
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para
Name = 'Kanaishk Cuckoo Search Dict-9.0.json'
Save_Often(CS_Dict,Name)'''

In [ ]:
print(Best_Score)
print(Best_Para)

In [29]:
def SKLEARN_CUCKOOSEARCH_PARAMGRID(Kmodel, Kparam_grid):
    lst = []
    population_size_lst = [50] #[30,50,10,50,70,90,20,30,40,60]
    pa_lst = [0.77] #[0.32,0.77,0.74,0.87,0.14,0.31,0.76,0.19,0.52,0.3]
    for i in range(1):#10
            
        Algo = basic.cs.CuckooSearch()
        Algo.set_parameters(
            population_size = population_size_lst[i],
            pa = pa_lst[i]
        )
        NIS = NatureInspiredSearchCV(
            estimator = Kmodel,
            param_grid = Kparam_grid,
            algorithm=Algo
        )
        lst.append(NIS)
    return lst

In [30]:
param_grid = {
    'n_estimators': range(50, 550, 50),
    'max_depth': [2, 4, 6, 8, 10, 20],
    'min_samples_split': range(2, 8, 2),
    'max_features': ["auto", "sqrt", "log2"]
}

In [31]:
NIS_lst = SKLEARN_CUCKOOSEARCH_PARAMGRID(model, param_grid)
print(len(NIS_lst))
Best_Score = 0
Best_Para = None 

1


In [32]:
CS_Dict = dict()
for i in range(len(NIS_lst)):
    Curr_Candidate = NIS_lst[i]
    Curr_Candidate.fit(X,y)
    Curr_Para = NIS_lst[i].algorithm.get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    CS_Dict[str(Curr_Para)] = Curr_Score
    print(Curr_Candidate.param_grid)
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para
Name = 'Kanaishk Cuckoo Search Dict-param_grid.json'
Save_Often(CS_Dict,Name)

Fitting at most 540 candidates
Optimization finished, 458 candidates were fitted
{'n_estimators': range(50, 550, 50), 'max_depth': [2, 4, 6, 8, 10, 20], 'min_samples_split': range(2, 8, 2), 'max_features': ['auto', 'sqrt', 'log2']}


In [33]:
print(Best_Score)
print(Best_Para)

0
{'population_size': 50, 'initialization_function': <function default_numpy_init at 0x000001E9D434ADE0>, 'individual_type': None, 'pa': 0.77}


In [19]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [23]:
clf = model.fit(X,y)

# Create our predictions
prediction = clf.predict(X)

# Create confusion matrix
confusion_matrix(y, prediction)

array([[    17,  11718],
       [     9, 117648]], dtype=int64)

In [24]:
# Display accuracy score
accuracy_score(y, prediction)

0.909368430814888

In [25]:
# Display F1 score
f1_score(y,prediction)

0.9525266877983022

In [17]:
#Orignal Paper Cuckoo Search: https://ieeexplore.ieee.org/abstract/document/5393690
#Sklearn Dir: https://github.com/timzatko/Sklearn-Nature-Inspired-Algorithms
#NIAPY ALgorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/algorithm.py
#NIAPY Cuckoo Search Algorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/basic/cs.py
#NIAPY How to test Algorithm: https://github.com/NiaOrg/NiaPy/blob/master/niapy/tests/test_algorithm.py
#NIAPY How to test CS: https://github.com/NiaOrg/NiaPy/blob/master/niapy/tests/test_cs.py
#NIAPY Task (Answer to Sphere Function): https://github.com/NiaOrg/NiaPy/blob/master/niapy/task.py